In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import skimage.io as io
from skimage import feature
from skimage.transform import resize
from sklearn.feature_selection import VarianceThreshold
from skimage.color import rgb2gray
from sklearn.decomposition import PCA
from sklearn.grid_search import GridSearchCV
from sklearn.svm import LinearSVC
from skimage.filters import gabor
from skimage.feature import corner_fast, corner_peaks, corner_harris
from sklearn.preprocessing import MinMaxScaler
import sys
import os
sys.path.append('../')
import helpers.histogram_classifier as hc

In [2]:
def sub_grid(image, size=32):
    """
    Split an image into a grid of non-overlapping square boxes, and return each box in turn
    :params image: image as numpy array
    :params size: side-length of box to use; should be a power of 2
    :return: yields each box in turn
    """
    pic_size = image.shape
    box_side = size
    for x in range(0, pic_size[0], box_side):
        for y in range(0, pic_size[1], box_side):
            yield image[x:x + box_side, y:y + box_side, :]

In [3]:
from IPython.core.display import HTML
css = open('styles/table-style.css').read() #+ open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [4]:
def get_image_features(img, box_size):
    """
    Get RGB and edge features for a single image by splitting the image into a grid of non-overlapping
    square boxes, and obtaining features for each box
    :params img: image as a numpy array
    :params box_size: the side-length in pixels of the box to be used
    :return: mean Red pixel value in each box
    :return: mean Green pixel value in each box
    :return: mean Blue pixel value in each box
    :return: number of edges in each box, extracted using skimage canny edges algorithm
    :return: number of corners in each box, extracted using skimage corner_fast algorithm
    """
    image_features = []
    for box in sub_grid(img, size=box_size):
        features = []
        features.append(np.mean(box[:, :, 0]))
        features.append(np.mean(box[:, :, 1]))
        features.append(np.mean(box[:, :, 2]))
        features.append(np.sum(feature.canny(rgb2gray(box), sigma=1)))
        features.append(corner_peaks(corner_fast(rgb2gray(box))).shape[0])
        image_features.append(np.array(features))
    image_features = np.array(image_features)
    return image_features[:, 0], image_features[:, 1], image_features[:, 2], image_features[:, 3], image_features[:, 4]

In [5]:
def get_complete_features(image_classes, image_names, box_size):
    """
    Get RGB and edge features for a dictionary of image names
    :params image_classes: list of n image classes
    :params image_names: dictionary of m image names for each class
    :params size: size as tuple for reshaping images for extracting edges and corners
    :return: red_features: average red pixel value for each box in each image
    :return: green_features: average green pixel value for each box in each image
    :return: blue_features: average blue pixel value for each box in each image
    :return: edge_features: number of edges detected for each box in each image
    :return: corner_features: number of corners detected for each box in each image
    :return: labels: array of true labels for each image
    """
    red_features = []
    green_features = []
    blue_features = []
    edge_features = []
    corner_features = []
    labels = []
    for c in image_classes:
        for i in image_names[c]:
            img = io.imread(os.path.join("../data/food-101/top_classes/", c, "", i))
            red, green, blue, edge, corners = get_image_features(img, box_size=box_size)
            red_features.append(red)
            green_features.append(green)
            blue_features.append(blue)
            edge_features.append(edge)
            corner_features.append(corners)
            labels.append(c)
    
    return np.array(red_features), np.array(green_features), np.array(blue_features), np.array(edge_features), np.array(corner_features), np.array(labels)

In [6]:
PCA_components = 256
BOX = 32

In [7]:
classes = pd.read_csv('../data/food-101/meta/top_classes.csv', index_col=0)
class_list = list(classes['class'].unique())

image_names = {}
for c in class_list:
    image_names[c] = np.array(classes[classes['class'] == c]['name'])

In [8]:
train, test = hc.split_data(image_names, 0.75)

for c in class_list:
    train[c] = train[c][:20]
    test[c] = test[c][:5]

In [9]:
#Get complete histogram features, edge and corners
all_red_features, all_green_features, all_blue_features, all_edge_features, all_corner_features, train_labels = get_complete_features(class_list, train, BOX)

In [10]:
#Get top features using PCA for red features
if all_red_features.shape[1] > PCA_components:
    red_pca = PCA(n_components=PCA_components)
    red_pca.fit(all_red_features)
    final_red_features = red_pca.transform(kept_red_features)
else:
    final_red_features = all_red_features

In [11]:
#Get top features using PCA for green features
if all_green_features.shape[1] > PCA_components:
    green_pca = PCA(n_components=PCA_components)
    green_pca.fit(all_green_features)
    final_green_features = green_pca.transform(all_green_features)
else:
    final_green_features = all_green_features

In [12]:
#Get top features using PCA for blue features
if all_blue_features.shape[1] > PCA_components:
    blue_pca = PCA(n_components=PCA_components)
    blue_pca.fit(all_blue_features)
    final_blue_features = blue_pca.transform(all_blue_features)
else:
    final_blue_features = all_blue_features

In [13]:
#Get top features using PCA for edge features
if all_edge_features.shape[1] > PCA_components:
    edge_pca = PCA(n_components=PCA_components)
    edge_pca.fit(all_edge_features)
    final_edge_features = edge_pca.transform(all_edge_features)
else:
    final_edge_features = all_edge_features

In [14]:
#Get top features using PCA for corner features
if all_corner_features.shape[1] > PCA_components:
    corner_pca = PCA(n_components=PCA_components)
    corner_pca.fit(all_corner_features)
    final_corner_features = corner_pca.transform(all_corner_features)
else:
    final_corner_features = all_corner_features

In [15]:
red_scaler = MinMaxScaler()
final_red_features = red_scaler.fit_transform(final_red_features)
green_scaler = MinMaxScaler()
final_green_features = green_scaler.fit_transform(final_green_features)
blue_scaler = MinMaxScaler()
final_blue_features = blue_scaler.fit_transform(final_blue_features)
edge_scaler = MinMaxScaler()
final_edge_features = edge_scaler.fit_transform(final_edge_features)
corner_scaler = MinMaxScaler()
final_corner_features = corner_scaler.fit_transform(final_corner_features)

In [16]:
all_features = np.concatenate((final_red_features, final_green_features, final_blue_features, final_edge_features, final_corner_features), axis=1)

In [17]:
#Setup grid-search for top parameters
c_vals = np.array([1e-5, 1e-3, 1e-1, 1, 10])
model = LinearSVC()
grid = GridSearchCV(estimator=model, param_grid=dict(C=c_vals), n_jobs=-1)

In [18]:
grid.fit(all_features, train_labels)

GridSearchCV(cv=None, error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([  1.00000e-05,   1.00000e-03,   1.00000e-01,   1.00000e+00,
         1.00000e+01])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [19]:
print("Best score = {}".format(grid.best_score_))
print("C: {}".format(grid.best_estimator_.C))

Best score = 0.30533333333333335
C: 0.001


In [20]:
%%time
svc = LinearSVC(C=grid.best_estimator_.C)
svc.fit(all_features, train_labels)

CPU times: user 4.5 s, sys: 44.6 ms, total: 4.55 s
Wall time: 4.55 s


In [21]:
#Get Test Features
all_red_features_t, all_green_features_t, all_blue_features_t, all_edge_features_t, all_corner_features_t, test_labels = get_complete_features(class_list, test, BOX)

In [22]:
if all_red_features_t.shape[1] > PCA_components:
    final_red_t = red_pca.transform(all_red_features_t)
else:
    final_red_t = all_red_features_t

if all_green_features_t.shape[1] > PCA_components:
    final_green_t = green_pca.transform(all_green_features_t)
else:
    final_green_t = all_green_features_t
    
if all_blue_features_t.shape[1] > PCA_components:
    final_blue_t = blue_pca.transform(all_blue_features_t)
else:
    final_blue_t = all_blue_features_t

if all_edge_features_t.shape[1] > PCA_components:
    final_edge_t = edge_pca.transform(all_edge_features_t)
else:
    final_edge_t = all_edge_features_t
    
if all_corner_features_t.shape[1] > PCA_components:
    final_corner_t = corner_pca.transform(all_corner_features_t)
else:
    final_corner_t = all_corner_features_t

In [23]:
red_scaler = MinMaxScaler()
final_red_t = red_scaler.fit_transform(final_red_t)
green_scaler = MinMaxScaler()
final_green_t = green_scaler.fit_transform(final_green_t)
blue_scaler = MinMaxScaler()
final_blue_t = blue_scaler.fit_transform(final_blue_t)
edge_scaler = MinMaxScaler()
final_edge_t = edge_scaler.fit_transform(final_edge_t)
corner_scaler = MinMaxScaler()
final_corner_t = corner_scaler.fit_transform(final_corner_t)

In [24]:
final_test_features = np.concatenate((final_red_t, final_green_t, final_blue_t, final_edge_t, final_corner_t), axis=1)

In [25]:
%%time
predicted_labels = svc.predict(final_test_features)

CPU times: user 14.9 ms, sys: 1.26 ms, total: 16.2 ms
Wall time: 10.2 ms


In [26]:
overall, metrics, confusion = hc.get_metrics(test_labels, predicted_labels, class_list)

In [27]:
overall

,Accuracy,F1,Precision,Recall
Results,0.304,0.284,0.297,0.304


In [28]:
metrics

,Precision,Recall,F1-Score,Support
pork_chop,0.37,0.34,0.36,250.0
lasagna,0.32,0.19,0.24,250.0
french_toast,0.25,0.18,0.21,250.0
guacamole,0.36,0.46,0.41,250.0
apple_pie,0.20,0.07,0.11,250.0
cheesecake,0.32,0.53,0.40,250.0
hamburger,0.27,0.19,0.23,250.0
fried_rice,0.35,0.24,0.29,250.0
carrot_cake,0.21,0.15,0.18,250.0
chocolate_cake,0.34,0.42,0.38,250.0


In [29]:
confusion

,pork_chop,lasagna,french_toast,guacamole,apple_pie,cheesecake,hamburger,fried_rice,carrot_cake,chocolate_cake,steak,pizza
pork_chop,86,10,15,8,3,11,6,11,16,18,32,34
lasagna,23,48,28,17,12,18,10,14,11,5,11,53
french_toast,17,26,45,13,6,16,10,13,12,28,18,46
guacamole,7,6,2,116,5,18,13,11,11,10,9,42
apple_pie,9,12,9,24,18,52,22,13,27,13,5,46
cheesecake,2,6,8,16,7,132,16,1,7,35,5,15
hamburger,5,11,14,23,14,24,48,18,17,12,4,60
fried_rice,10,8,11,44,8,20,8,61,11,8,3,58
carrot_cake,17,5,10,20,9,54,23,9,38,24,5,36
chocolate_cake,15,4,16,10,3,40,6,1,14,106,16,19
